# The aHB circuit tracks fictive trajectories

We have seen in the previous notebooks that the aHB integrator could be capabable of keep track of past motion over a significant amount of time. In this notebook, we will see how much it can keep track of past trajectories over significant amounts of time.

## A first look at cumulative directional motion and network phase

In [ ]:
%matplotlib widget
from pathlib import Path

import lotr.plotting as pltltr
import numpy as np
import pandas as pd
from bouter.utilities import crop
from lotr import A_FISH, DATASET_LOCATION, LotrExperiment
from lotr.behavior import get_bouts_props_array
from lotr.default_vals import REGRESSOR_TAU_S
from lotr.utils import convolve_with_tau, linear_regression, nan_phase_jumps
from matplotlib import pyplot as plt
from scipy.stats import pearsonr, spearmanr, wilcoxon
from tqdm import tqdm

COLS = pltltr.COLS

In [ ]:
exp = LotrExperiment(DATASET_LOCATION / "210601_f0" / "210601_f0_natmov_noeyes")

In [ ]:
bias_arr = get_bouts_props_array(exp.n_pts, exp.bouts_df)

In [ ]:
t_lims = (900, 1500)  # (1500, 1900)  #
f, axs = plt.subplots(
    2,
    1,
    figsize=(5, 2),
    gridspec_kw=dict(hspace=0.1, height_ratios=[0.3, 1], bottom=0.2),
)

for i in exp.bouts_df.index:
    if (exp.bouts_df.loc[i, "t_start"] > t_lims[0]) & (
        exp.bouts_df.loc[i, "t_start"] < t_lims[1]
    ):
        for d in ["lf", "rt", "fw"]:
            if exp.bouts_df.loc[i, "direction"] == d:
                axs[0].axvline(
                    exp.bouts_df.loc[i, "t_start"],
                    c=COLS["sides"][d],
                    lw=0.0,
                    # label="left bouts" if leftlab else "__nolegend__",
                )
            # leftlab = False
t_sel = (exp.behavior_log["t"] > t_lims[0]) & (exp.behavior_log["t"] < t_lims[1])
axs[0].plot(
    exp.time_arr,  # exp.behavior_log.loc[t_sel, "t"],
    bias_arr,  # exp.behavior_log.loc[t_sel, "tail_sum"],
    c=".5",
    lw=1.5,
    rasterized=True,
)

axs[1].imshow(
    exp.traces[:, exp.hdn_indexes][:, np.argsort(exp.rpc_angles)].T,
    cmap=COLS["dff_plot"],
    aspect="auto",
    vmax=1.4,
    vmin=-1.4,
    extent=(0, exp.time_arr[-1], -np.pi, np.pi),
)

phase = exp.network_phase.copy()
phase[1:][np.abs(np.diff(phase)) > np.pi] = np.nan
axs[1].plot(exp.time_arr, phase, c=COLS["ph_plot"])
axs[0].set(
    xlim=t_lims,
    ylim=(-0.5, 0.5),
    xticklabels=[],
    ylabel="Angle \n turned",
    **pltltr.get_pi_labels(coefs=[-0.125, 0, 0.125], ax="y")
)
axs[1].set(
    xlim=t_lims,
    xlabel="Time (s)",
    ylabel="Phase (rad)",
    **pltltr.get_pi_labels(coefs=[-0.5, 0, 0.5], ax="y")
)
pltltr.despine(axs[0], ["top", "right", "bottom"])
pltltr.despine(axs[0])

pltltr.savefig("multiple_turns", folder="supplementary/S2")